In [26]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from spp.SpatialPyramidPooling import SpatialPyramidPooling
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from os import makedirs
from os.path import expanduser, exists, join

In [9]:
batch_size = 16
img_width, img_height = 256, 256
num_classes = 2
n_epoches = 100
input_shape = (img_height, img_width, 3)

In [10]:
train_data_dir = "../data/train"
valid_data_dir = "../data/valid"

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                featurewise_center=False,  # set input mean to 0 over the dataset
                                samplewise_center=True,  # set each sample mean to 0
                                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                samplewise_std_normalization=True,  # divide each input by its std
#                                 zca_whitening=False,  # apply ZCA whitening
#                                 rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
                                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                horizontal_flip=True,  # randomly flip images
                                vertical_flip=True) # randomly flip images                                

train_generator0 = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    seed=10)
train_generator1 = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height*2, img_width*2),
                                                    batch_size=batch_size,
                                                    seed=10)
train_generator2 = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height*4, img_width*4),
                                                    batch_size=batch_size,
                                                    seed=10)

val_datagen = ImageDataGenerator(rescale=1. / 255)

valid_generator = val_datagen.flow_from_directory(valid_data_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                seed=10)

print ("train data: ", train_generator0.n)
print ("train data: ", train_generator1.n)
print ("train data: ", train_generator2.n)
print ("valid data: ", valid_generator.n)

Found 204 images belonging to 2 classes.
Found 204 images belonging to 2 classes.
Found 204 images belonging to 2 classes.
Found 46 images belonging to 2 classes.
('train data: ', 204)
('train data: ', 204)
('train data: ', 204)
('valid data: ', 46)


In [20]:
class MixIterator(object):
    def __init__(self, iters):
        self.iters = iters
        self.N = sum([it.n for it in self.iters])
        
    def reset(self):
        for it in self.iters: it.reset()
            
    def __iter__(self):
        return self
    
    def next(self, *args, **kwargs):
        nexts = [next(it) for it in self.iters]
        n0 = np.concatenate([n[0] for n in nexts])
        n1 = np.concatenate([n[1] for n in nexts])
        n2 = np.concatenate([n[2] for n in nexts])
        return (n0,n1,n2)

mi = MixIterator([train_generator0, train_generator1,train_generator2])

In [45]:
def create_baseline():
    # create model  
    model = Sequential()
    BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=input_shape, padding = 'same'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(128, kernel_size=(3, 3),activation='relu', padding = 'same'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(512, (3, 3), activation='relu', padding = 'same'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))  
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(num_classes, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(SpatialPyramidPooling([1, 2, 4]))
#     model.add(GlobalAveragePooling2D())
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [46]:
base_model = create_baseline()
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
batch_normalization_68 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 256, 256, 64)      18496     
_________________________________________________________________
batch_normalization_69 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 256, 256, 128)     73856     
_________________________________________________________________
batch_normalization_70 (Batc (None, 256, 256, 128)     512       
_________________________________________________________________
conv2d_70 (Conv2D)           (None, 256, 256, 256)     295168    
__________

In [47]:
base_model.compile(Adam(lr=.001, decay=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
base_model.save_weights("jbm_custom_weights_15.h5")

In [ ]:
base_model.fit_generator(mi,
                      steps_per_epoch = train_generator0.n // batch_size,
                      validation_data = valid_generator,
                      validation_steps = valid_generator.n // batch_size,
                      epochs = n_epoches,
                      verbose = 1)

Epoch 1/100


In [ ]:
base_model.save_weights("jbm_inception_weights_5.h5")